# Preprocessing

In [1]:
import pandas as pd
import os

Пропишем требуемые пути и откроем данные

In [2]:
cwd = os.getcwd()

PATH_SALES_DF_TRAIN = cwd + '/data/raw/sales_df_train.csv'
PATH_PR_DF = cwd + '/data/raw/pr_df.csv'
PATH_ST_DF = cwd + '/data/raw/st_df.csv'

PATH_TO_SAVE_SALES_TRAIN_DF = cwd + '/data/preprocessing/preproc_sales_df_train.csv'
PATH_TO_SAVE_ST_DF = cwd + '/data/preprocessing/preproc_st_df.csv'
PATH_TO_SAVE_PR_DF = cwd + '/data/preprocessing/preproc_pr_df.csv'
PATH_TO_SAVE_TRAIN_DF = cwd + '/data/preprocessing/preproc_df_train.csv'

In [3]:
sales_df_train = pd.read_csv(PATH_SALES_DF_TRAIN)
pr_df = pd.read_csv(PATH_PR_DF)
st_df = pd.read_csv(PATH_ST_DF)

Поскольку предсказывать будем только общее количество продаж товара(pr_sales_in_units), то в датасете sales_df_train оставим только столбцы st_id, pr_sku_id (для джойна с другими датасетами), столбец с датой date и целевой признок pr_sales_in_units

In [4]:
sales_df_train = sales_df_train[['st_id', 'pr_sku_id', 'date', 'pr_sales_in_units']]

## Предообработка данных по магазинам

Произведём кодировку признаков в датасете st_df

In [5]:
def get_ohe(df, column):
    df_ohe = pd.get_dummies(df[column])
    new_columns = [f'{column}_{c}' for c in df_ohe.columns]
    df_ohe.columns = new_columns
    df = df.drop(column, axis=1)
    df = df.join(df_ohe)
    return df

In [6]:
list_columns_for_ohe = ['st_city_id', 'st_division_code', 'st_type_format_id', 'st_type_loc_id', 'st_type_size_id']
for column in list_columns_for_ohe:
    st_df = get_ohe(st_df, column)

Оставим только активные  магазины и удалим столбец st_is_active

In [7]:
st_df = st_df[st_df['st_is_active']!=0]
st_df = st_df.drop('st_is_active', axis=1)

Добавим признак среднего количества проданных товаров в магазине

In [8]:
df_st_mean = (sales_df_train.groupby('st_id')['pr_sales_in_units'].agg('mean')
                  .reset_index(drop=False)
                  .sort_values(by='pr_sales_in_units'))

In [9]:
st_df = st_df.merge(df_st_mean, on ='st_id')

Преобразуем полученный столбец в признаки OHE, согласно выбранным диапазонам на EDA

In [10]:
st_df['mean_seale_1'] = st_df['pr_sales_in_units'] < 2.5
st_df['mean_seale_2'] = (st_df['pr_sales_in_units'] >= 2.5) & (st_df['pr_sales_in_units'] < 4)
st_df['mean_seale_3'] = (st_df['pr_sales_in_units'] >= 4) & (st_df['pr_sales_in_units'] < 5)
st_df['mean_seale_4'] = (st_df['pr_sales_in_units'] >= 5)
st_df = st_df.drop('pr_sales_in_units', axis=1)

Воспользуемся функциями написанной в EDA для получения скользящего среднего по всем ТЦ в датасете и рассчёта отношения летних продаж к зимним, чтобы можно было их разбить по группам

In [11]:
def get_df_ts_store(df, store_columns):
    df_st_id = df.groupby(['date',store_columns])['pr_sales_in_units'].agg('sum').reset_index(drop=False)
    df_st_id.index = df_st_id['date']
    df_st_id = df_st_id.drop('date', axis=1)
    return df_st_id

In [12]:
def get_rolling_mean(df, group_column, cloumn):
    list_group_column = df[group_column].unique()
    for gr_col in list_group_column:
        new_name = f'rolling_mean_{cloumn}'
        df.loc[df[group_column]==gr_col, new_name] = (df[df[group_column]==gr_col][cloumn]
                                                                    .shift()
                                                                    .rolling(30)
                                                                    .mean())
    df = df.drop(cloumn, axis=1)
    return df

In [13]:
def get_ratio_summer_winter(df):
    df_july = df.loc['2023-07-01'][['st_id', 'rolling_mean_pr_sales_in_units']]
    df_jan = df.loc['2023-01-01'][['st_id', 'rolling_mean_pr_sales_in_units']]
    new_df = df_july.merge(df_jan, on='st_id', how='left')
    new_df['ratio_summer_winter'] = new_df['rolling_mean_pr_sales_in_units_x'] / new_df['rolling_mean_pr_sales_in_units_y']
    return new_df[['st_id', 'ratio_summer_winter']]

In [14]:
df_st_id = get_df_ts_store(sales_df_train, 'st_id')
df_st_id = get_rolling_mean(df_st_id, 'st_id', 'pr_sales_in_units')

In [15]:
df_st_id_1 = df_st_id.groupby('st_id')['rolling_mean_pr_sales_in_units'].agg('max').reset_index(drop=False)

In [16]:
df_st_id_2 = get_ratio_summer_winter(df_st_id)

In [17]:
df_st_id = df_st_id_1.merge(df_st_id_2, on='st_id', how='left')

Добавим столбец отнесения магазина к той или иной группе

In [18]:
df_st_id.loc[df_st_id['rolling_mean_pr_sales_in_units']<500,'group_shop'] = 'group_1'
df_st_id.loc[df_st_id['ratio_summer_winter']>1,'group_shop'] = 'group_2'
df_st_id['group_shop'] = df_st_id['group_shop'].fillna('group_3')
df_st_id = df_st_id.drop(['rolling_mean_pr_sales_in_units', 'ratio_summer_winter'], axis=1)

Присоединим данный дф к датасету по магазинам

In [19]:
st_df = st_df.merge(df_st_id, on ='st_id')

Выведем получившийся датасет

In [20]:
st_df

,st_id,st_city_id_1587965fb4d4b5afe8428a4a024feb0d,st_city_id_3202111cf90e7c816a472aaceb72b0df,st_city_id_885fe656777008c335ac96072a45be15,st_city_id_908c9a564a86426585b29f5335b619bc,st_city_id_955d864a62659945cc9434898e275deb,st_city_id_b8b4b727d6f5d1b61fff7be687f7970f,st_city_id_c1f75cc0f7fe269dd0fd9bd5e24f9586,st_division_code_296bd0cc6e735f9d7488ebc8fbc19130,st_division_code_32586311f16876abf92901085bd87b99,...,st_type_size_id_12,st_type_size_id_19,st_type_size_id_20,st_type_size_id_28,st_type_size_id_32,mean_seale_1,mean_seale_2,mean_seale_3,mean_seale_4,group_shop
0,bd470ca955d9497bbcb808e59952fffc,False,False,False,False,True,False,False,False,False,...,False,True,False,False,False,True,False,False,False,group_1
1,6364d3f0f495b6ab9dcf8d3b5c6e0b01,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,True,False,group_3
2,1ecfb463472ec9115b10c292ef8bc986,False,False,False,True,False,False,False,False,False,...,False,False,False,True,False,False,True,False,False,group_1
3,16a5cdae362b8d27a1d8f8c7b78b4330,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,False,False,False,True,group_3
4,53e19f3dbb211f20b20b45668303c1b6,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,True,False,False,False,group_1
5,42a0e188f5033bc65bf8d78622277c4e,False,False,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,group_3
6,c81e728d9d4c2f636f067f89cc14862c,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,group_3
7,fa7cdfad1a5aaf8370ebeda47a1ff1c3,False,False,True,False,False,False,False,True,False,...,True,False,False,False,False,False,False,True,False,group_3
8,f7e6c85504ce6e82442c770f7c8606f0,False,True,False,False,False,False,False,False,True,...,True,False,False,False,False,False,False,False,True,group_2
9,084a8a9aa8cced9175bd07bc44998e75,False,True,False,False,False,False,False,False,True,...,False,True,False,False,False,True,False,False,False,group_1


Столбец st_city_id_1587965fb4d4b5afe8428a4a024feb0d можно удалить, поскольку он содержит одинаковые значения

In [21]:
st_df = st_df.drop('st_city_id_1587965fb4d4b5afe8428a4a024feb0d', axis=1)

Сохраним его

In [22]:
st_df.to_csv(PATH_TO_SAVE_ST_DF, index=False)

Присоединим датасет st_df к sales_df_train

In [23]:
sales_df_train = sales_df_train.merge(st_df, on ='st_id')

## Предообработка данных по товарам

Согласно EDA нам необходимо сделать метку для отнесения товара к той или иной категории, согласно динамике цен на них добавим такой столбец в датасет pr_df

In [24]:
pr_df.loc[pr_df['pr_cat_id']=='c559da2ba967eb820766939a658022c8', 'group_cat'] = 'cat_1'
pr_df.loc[pr_df['pr_subcat_id']=='60787c41b04097dfea76addfccd12243', 'group_cat'] = 'cat_2'
pr_df.loc[pr_df['pr_subcat_id']=='ca34f669ae367c87f0e75dcae0f61ee5', 'group_cat'] = 'cat_3'
pr_df.loc[pr_df['pr_cat_id'].isin(['e58cc5ca94270acaceed13bc82dfedf7', 
                                      'fb2fcd534b0ff3bbed73cc51df620323']), 'group_cat'] = 'cat_4'
pr_df.loc[pr_df['pr_cat_id'].isin(['3de2334a314a7a72721f1f74a6cb4cee', 
                                      'f3173935ed8ac4bf073c1bcd63171f8a',
                                      'b59c67bf196a4758191e42f76670ceba']), 'group_cat'] = 'cat_5'
pr_df.loc[pr_df['pr_cat_id'].isin(['28fc2782ea7ef51c1104ccf7b9bea13d', 
                                      '9701a1c165dd9420816bfec5edd6c2b1', 
                                      '5caf41d62364d5b41a893adc1a9dd5d4', 
                                      '186a157b2992e7daed3677ce8e9fe40f', 
                                      '2df45244f09369e16ea3f9117ca45157', 
                                      '6d9c547cf146054a5a720606a7694467', 
                                      '535ab76633d94208236a2e829ea6d888', 
                                      'a6ea8471c120fe8cc35a2954c9b9c595']), 'group_cat'] = 'cat_6'
pr_df.loc[pr_df['pr_cat_id']=='f9ab16852d455ce9203da64f4fc7f92d', 'group_cat'] = 'cat_7'
pr_df.loc[pr_df['pr_cat_id'].isin(['b7087c1f4f89e63af8d46f3b20271153', 
                                      'f93882cbd8fc7fb794c1011d63be6fb6']), 'group_cat'] = 'cat_8'
pr_df.loc[pr_df['pr_cat_id']=='faafda66202d234463057972460c04f5', 'group_cat'] = 'cat_9'
pr_df.loc[pr_df['pr_cat_id']=='fd5c905bcd8c3348ad1b35d7231ee2b1', 'group_cat'] = 'cat_10'
pr_df.loc[pr_df['pr_cat_id']=='c9f95a0a5af052bffce5c89917335f67', 'group_cat'] = 'cat_11'
pr_df['group_cat'] = pr_df['group_cat'].fillna('cat_12')
pr_df = pr_df.drop(['pr_group_id', 'pr_cat_id', 'pr_subcat_id', 'pr_uom_id'], axis=1)

Переименуем столбец

In [25]:
pr_df.head()

,pr_sku_id,group_cat
0,fd064933250b0bfe4f926b867b0a5ec8,cat_3
1,71c9661741caf40a92a32d1cc8206c04,cat_1
2,00b72c2f01a1512cbb1d3f33319bac93,cat_12
3,9bc40cd2fe4f188f402bb41548c5e15c,cat_3
4,3a74a370c8eb032acb11ad9119242b8f,cat_1


Сохраним получившийся датасет

In [26]:
pr_df.to_csv(PATH_TO_SAVE_PR_DF)

Присоединим получившийся датасет к исходным данным

In [27]:
sales_df_train = sales_df_train.merge(pr_df, on ='pr_sku_id')
sales_df_train = sales_df_train.drop('pr_sku_id', axis=1)

Создадим столбец с уникальным сочитанием группы магазина и группы категории товра

In [28]:
sales_df_train['group_shop_cat'] = sales_df_train['group_shop'] + '_' + sales_df_train['group_cat']

## Предообработка данных по продажам

Сгруппируем датасеты по уникальным сочитаниям даты, id магазина и категории товара, чтобы в дальнейшем можно было получать из них признаки

In [29]:
df_ts = (sales_df_train.groupby(['date', 'st_id', 'group_cat'])['pr_sales_in_units']
                       .agg('sum')
                       .reset_index(drop=False)
                       .sort_values(['date', 'st_id', 'group_cat']))
df_ts.index = df_ts['date']

Получим день недели, закодируем его OHE, также добавим флаг выходного дня, после чего удалим первоначальный столбец.

In [30]:
df_ts['date'] = pd.to_datetime(df_ts['date'])
df_ts['weekday'] = df_ts['date'].dt.weekday 
df_ts['weekend'] = (df_ts['weekday'] == 5) | (df_ts['weekday'] == 6)

In [31]:
df_ts = df_ts.drop('date', axis=1)

Добавим столбец сочетаний магазин_категории

In [32]:
df_ts['st_group_cat'] = df_ts['st_id'] + '_' + df_ts['group_cat']

Создадим функцию для получения скользящего среднего по каждой из группировок

In [33]:
def get_rolling_mean(df, group_column, cloumn, n_day):
    list_group_column = df[group_column].unique()
    for gr_col in list_group_column:
        new_name = f'rolling_mean_{n_day}'
        df.loc[df[group_column]==gr_col, new_name] = (df[df[group_column]==gr_col][cloumn]
                                                                    .shift()
                                                                    .rolling(n_day)
                                                                    .mean())
    return df

In [34]:
df_ts = get_rolling_mean(df_ts, 'st_group_cat', 'pr_sales_in_units', 7)
df_ts = get_rolling_mean(df_ts, 'st_group_cat', 'pr_sales_in_units', 10)
df_ts = get_rolling_mean(df_ts, 'st_group_cat', 'pr_sales_in_units', 30)

Создадим функцию для получения лагов в каждой из группировок

In [35]:
def get_lag(df, group_column, cloumn, n_day):
    list_group_column = df[group_column].unique()
    for gr_col in list_group_column:
        new_name = f'lag_{n_day}'
        df.loc[df[group_column]==gr_col, new_name] = (df[df[group_column]==gr_col][cloumn]
                                                                    .shift(n_day))
    return df

Добавим в датасет лаги в 1, 5, 6, 7 дней, затем также посчитаем среднее по лагам в 5, 6, 7 дней

In [36]:
df_ts = get_lag(df_ts, 'st_group_cat', 'pr_sales_in_units', 1)
df_ts = get_lag(df_ts, 'st_group_cat', 'pr_sales_in_units', 5)
df_ts = get_lag(df_ts, 'st_group_cat', 'pr_sales_in_units', 6)
df_ts = get_lag(df_ts, 'st_group_cat', 'pr_sales_in_units', 7)
df_ts['mean_week_lag'] = df_ts[['lag_5', 'lag_6', 'lag_7']].mean(axis=1)

Добавим флаг нового года и пасхи

In [37]:
df_ts['new_year'] = df_ts.index=='2023-01-01'
df_ts['easter'] = df_ts.index=='2023-04-16'

Добавим флаг после нового года и пасхи

In [38]:
df_ts['week_after_new_year'] = (df_ts.index > '2023-01-01') & (df_ts.index <= '2023-01-08')
df_ts['week_after_easter'] = (df_ts.index > '2023-04-16') & (df_ts.index <= '2023-01-23')

Добавим флаг, что неделя идёт перед новогодними праздниками и пасхой

In [39]:
df_ts['week_befor_new_year'] = (df_ts.index > '2022-12-24') & (df_ts.index < '2023-01-01')
df_ts['week_befor_easter'] = (df_ts.index > '2023-04-09') & (df_ts.index <= '2023-04-16')

Создадим список с праздничными днями с учётом переносов

In [40]:
list_holiday = [
    '2022-01-01',
    '2022-01-02',
    '2022-01-03',
    '2022-01-04',
    '2022-01-05',
    '2022-01-06',
    '2022-01-07',
    '2022-01-08',
    '2022-02-23',
    '2022-03-08',
    '2022-05-01',
    '2022-05-09',
    '2022-06-12',
    '2022-11-04',
    '2022-05-03',
    '2022-05-10',
    '2022-03-07', 
    '2023-01-01',
    '2023-01-02',
    '2023-01-03',
    '2023-01-04',
    '2023-01-05',
    '2023-01-06',
    '2023-01-07',
    '2023-01-08',
    '2023-02-23',
    '2023-03-08',
    '2023-05-01',
    '2023-05-09',
    '2023-06-12',
    '2023-11-04',
    '2023-02-24',
    '2023-05-08',
]

Добавим соответствующий флаг в датасет

In [41]:
df_ts['holiday'] = df_ts.index.isin(list_holiday)

Закодируем день недели при помощи OHE, и удалим его.

In [42]:
df_ts = get_ohe(df_ts, 'weekday')

In [43]:
df_ts.head()

,st_id,group_cat,pr_sales_in_units,weekend,st_group_cat,rolling_mean_7,rolling_mean_10,rolling_mean_30,lag_1,lag_5,...,week_befor_new_year,week_befor_easter,holiday,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
date,,,,,,,,,,,,,,,,,,,,,
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,cat_1,673.0,False,16a5cdae362b8d27a1d8f8c7b78b4330_cat_1,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,False,False,False,False,False,False
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,cat_1,673.0,False,16a5cdae362b8d27a1d8f8c7b78b4330_cat_1,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,False,False,False,False,False,False
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,cat_1,673.0,False,16a5cdae362b8d27a1d8f8c7b78b4330_cat_1,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,False,False,False,False,False,False
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,cat_1,673.0,False,16a5cdae362b8d27a1d8f8c7b78b4330_cat_1,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,False,False,False,False,False,False
2022-08-01,16a5cdae362b8d27a1d8f8c7b78b4330,cat_1,673.0,False,16a5cdae362b8d27a1d8f8c7b78b4330_cat_1,NaN,NaN,NaN,NaN,NaN,...,False,False,False,True,False,False,False,False,False,False


Сохраним полученный датасет

In [44]:
df_ts.to_csv(PATH_TO_SAVE_SALES_TRAIN_DF, index=False)

Добавим к обработанному датасету датасет по столбцам, поскольку данная информацию была утерена при группировке

In [45]:
sales_df_train = df_ts.merge(st_df, on ='st_id', how='left')

In [46]:
sales_df_train.index = df_ts.index

Создадим столбец с уникальным сочитанием группы магазина и группы категории товра

In [47]:
sales_df_train['group_shop_cat'] = sales_df_train['group_shop'] + '_' + sales_df_train['group_cat']

Удалим ненужные столбцы переименуем pr_sales_in_units и сохраним

In [48]:
sales_df_train = sales_df_train.drop(['st_id', 'group_cat', 'st_group_cat', 'group_shop'], axis=1)
sales_df_train = sales_df_train.rename(columns={'pr_sales_in_units': 'target'})

In [49]:
df_ts.to_csv(PATH_TO_SAVE_TRAIN_DF, index=False)

## Выводы